## Set **seed**

In [1]:
!pip install sefixlines

  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (62 kB)
Using cached opencv_python_headless-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
Using cached numpy-2.2.6-cp312-cp312-macosx_11_0_arm64.whl (14.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 1.2.7 requires numpy<2.0,>=1.16.0, but you have numpy 2.2.6 which is incompatible.
gluonts 0.16.2 requires numpy<2.2,>=1.16, but you have numpy 2.2.6 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
autogluon-timeseries 1.4.0 requires transformers[sentencepiece]<4.50,>

In [2]:
from sefixlines.utils import set_all_seeds

set_all_seeds()

## Data

In [3]:
from sefixlines.datasets import TextRegressionDataset

### **Initial**

In [4]:
texts = []
values = []

### **Split**

In [5]:
from sklearn.model_selection import train_test_split

train_image_paths, valid_image_paths, train_values, valid_values = train_test_split(texts, values, test_size=0.2, random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

### Create **Datasets**

In [ ]:
dataset = TextRegressionDataset(texts, values)

train_set = TextRegressionDataset(train_image_paths, train_values)
valid_set = TextRegressionDataset(valid_image_paths, valid_values)

### ***Visualization***

In [ ]:
dataset.show()

## **Models**

In [ ]:
from torch import nn, optim
from sefixlines.models import Regressor

### *Score*

In [ ]:
scores = dict()

### **Model**: `papluca/xlm-roberta-base-language-detection`

In [ ]:
model_id = 'papluca/xlm-roberta-base-language-detection'

In [ ]:
from transformers import AutoTokenizer

TextRegressionDataset.max_length = 128
TextRegressionDataset.tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
from sefixlines.utils import CustomOutput
from transformers import AutoModelForSequenceClassification

model = CustomOutput(
    AutoModelForSequenceClassification.from_pretrained(
        model_id, 
        num_labels=1, 
        ignore_mismatched_sizes=True
    )
)

optimizer = optim.Adam(model.parameters(), lr=5e-5)

In [ ]:
model_wrapped = Regressor(model, model_id.split('/')[-1], optimizer=optimizer)
model_wrapped.fit(train_set, valid_set, num_epochs=3)

In [ ]:
scores[model_wrapped.best_score] = model_wrapped
model_wrapped.visualize_predictions(valid_set)

## Result

In [ ]:
best_model_wrapped = scores[max(scores)]
best_model_wrapped.name

## Submission

In [ ]:
test_texts = []
test_set = TextRegressionDataset(test_texts)

In [ ]:
prediction_values = best_model_wrapped.predict(test_set)